# Generate data for `test_correct_count_rate`

In [ ]:
import importlib
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer.buildfgssteps import BuildFGSSteps, shift_to_id_attitude

In [ ]:
def find_package_location(package):
    return importlib.util.find_spec(package).submodule_search_locations[0]


def find_repo_location(package):
    return os.path.abspath(os.path.join(find_package_location(package), os.pardir))

In [ ]:
repo_location = find_repo_location(package='jwst_magic')
__location__ = os.path.join(repo_location, 'jwst_magic', 'tests')
ROOT = 'test_buildfgssteps'
SEGMENT_INFILE = os.path.join(__location__, 'data', 'unshifted_all_found_psfs_{}_G1.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))
CENTER_POINTING_1 = os.path.join(__location__, 'data', 'center_pointing_{}_G1.txt'.format(ROOT))
TEST_DIRECTORY = os.path.join(__location__, 'out', ROOT)

In [ ]:
with fits.open( os.path.join(__location__, 'data', 'fgs_data_2_cmimf.fits')) as hdulist:
    open_image = hdulist[1].data

In [ ]:
d = {}
for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 10
        else:
            n = 1
            
        for i in range(n):
            # Run the code
            fgs_im, guiding_selections_file, psf_center_file = shift_to_id_attitude(
                    open_image, ROOT, guider, TEST_DIRECTORY, guiding_selections_file=SELECTED_SEGS,
                    all_found_psfs_file=SEGMENT_INFILE, center_pointing_file=CENTER_POINTING_1,
            psf_center_file=None, crowded_field=False, logger_passed=True)
            BFS = BuildFGSSteps(fgs_im, guider, ROOT, step, guiding_selections_file=guiding_selections_file,
                                out_dir=TEST_DIRECTORY, shift_id_attitude=True)

            # Don't know how to test bias - since it is so variable
            if step is not 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step is 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , step, 'countrate']):
            if (l != [] and step == 'LOSTRK') or (name in ['countrate', 'norm']):
                d[guider][step][name] = l[0]
            elif l != [] and name != 'countrate':
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]

        

In [ ]:
pprint.pprint(d)

# Generate data for `coordinate_tranforms`

In [ ]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [ ]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        elif 'detector' in arg_names:
            result = func_to_call(x, y, 'NRCA3')
            print(result)
        else:
            result = func_to_call(x, y)
            print(result)

In [ ]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)